# Drowsiness Detection using OpenCV and Tensorflow w/ Keras
### By Dustin Reyes
### Highlights 
1. Classification of whether an eye is closed or opened
2. The model can correctly classify whether the eyes of a person is open or close in under the ff circumstances: 
    * Under the different lighting environments (Dark or Light)
    * Whether a person wearing glasses or not. (Addresses lens glare)
    * Real time eye status tracking.
3. A combination of Computer Vision and Deep learning models for image classification was done for real time drowsiness detection.
4. The image classification model achieved 95% Average Test Accuracy across 250 epochs.
5. The project tackled the importance of Grayscale images in doing realtime classification tasks.
6. The use of facial recognition algorithms such as the Haar Cascade Algorithm was used for both face and eye detection.

### Data Source: http://parnec.nuaa.edu.cn/xtan/data/ClosedEyeDatabases.html

# 1. Executive Summary

The purpose of this notebook is to list the details regarding the Drowsiness Detection using a combination of OpenCV and Tensorflow w/ Keras. The project is divided into two main parts: Training a convolutional neural network model which can correctly classify whether the eyes of a person is open or close and using the trained model to provide real-time video classification about the user's state on whether he/she is drowsy. A threshold score is implemented which identifies that the user is drowsy with the score incrementing one step at a time as long as the eyes are closed and with the score decrementing if the eyes are then opened. An alarm system was also created to alert the user that he/she is already drowsy and is in danger of falling asleep. The model used for eye state classification achieved an average of **95% test accuracy**. The goal of the project is to detect drowsiness which will serve to reduce drowsy driving and unproductivity due to drowsiness at the workplace. By preventing the occurence of these scenarios, the project will surely help in the reduction of events related to drowsiness.

The project however has its own limitations and these are. 
* The first limitation is the hardware constraints since this was only using a builtin webcam. Newer and particularly higher end webcams can provide far better results as the frame rates can be better. 
* The second limitation is that since the CNN model assumes that it is trained on a singular user, the project is thus limited to only one person/user at a time. 
* The third is that the availability of sample data is somewhat limited, therefore poses more room for improvement.

Improvements for the future include:
* Training more images
* Using better hardware (e.g. PC or webcam)
* Training images regardless of ethnicity or race.

# 2. Data Description

Drowsiness, one of the leading causes of car accidents in the world, has been a major problem of countries who value productivity throughout the day. The risk, danger, and often tragic results of drowsy driving are alarming. By definition, drowsy driving is a combination of driving and sleepiness or fatigue. This scenario often happens when the driver has insufficient sleep. However, this may also happen to drivers with chronic sleep disorders and those who are under the influence of alcohol. 

The data used for this project was taken from http://parnec.nuaa.edu.cn/xtan/data/ClosedEyeDatabases.html which contains **2433** subjects, among which **1192 subjects have both eyes closed**, and **1231 subjects with eyes open**. The dataset consists of the whole face of two main resolutions (the original raw images and the resized 100 x 100 pixels) and the extracted eyepatches (size 24 x 24 pixels) centered at the localized eye position. 

The figure below shows some examples:

<img src="Eyes.jpg" width="700">

##  Copying images to training, validation, and test directories
#### Important Note: 
The contents of this ntbk is derived from the results of a separate jupyter ntbk entitled "Training_script_eyesonly (Individual Project)" This notebook only details the code used and is not meant to run it as it will take considerable time to display the results.

In [1]:
import os
import shutil
import random
import keras
import numpy as np
import matplotlib.pyplot as plt

from keras.utils.np_utils import to_categorical
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dropout, Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model
from tensorflow.keras import models
from tensorflow.keras import layers

Using TensorFlow backend.


#### Important Note: 
The images were extracted from a zipped file and was contained in two separate folders: closed and opened. The file contents were then renamed accordingly and the contents were merged together. The file "extractor.ipynb" was used to do the file and directory manipulations as the training process was done in Jojie. 

In [ ]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = '/home/msds2020/dreyes/Machine Learning 2.0/Individual Project/Train_data3'

# The directory where we will store our smaller dataset
# The immediate subsequent line of code creates a directory
# named "project indiv2" with a folder name "results" for storage
base_dir = os.path.expanduser('~') + '/project_indiv2/results/'
os.makedirs(base_dir, exist_ok=True)

# Directories for our training,validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir, exist_ok=True)

# Directory with our training opened eye pictures
train_opened_dir = os.path.join(train_dir, 'opened')
os.makedirs(train_opened_dir, exist_ok=True)

# Directory with our training closed eye pictures
train_closed_dir = os.path.join(train_dir, 'closed')
os.makedirs(train_closed_dir, exist_ok=True)

# Directory with our validation opened eye pictures
validation_opened_dir = os.path.join(validation_dir, 'opened')
os.makedirs(validation_opened_dir, exist_ok=True)

# Directory with our validation closed eye pictures
validation_closed_dir = os.path.join(validation_dir, 'closed')
os.makedirs(validation_closed_dir, exist_ok=True)

# Directory with our test opened eye pictures
test_opened_dir = os.path.join(test_dir, 'opened')
os.makedirs(test_opened_dir, exist_ok=True)

# Directory with our test closed pictures
test_closed_dir = os.path.join(test_dir, 'closed')
os.makedirs(test_closed_dir, exist_ok=True)

# Copy first 600 opened eye images to train_opened_dir
fnames = ['opened.{}.jpg'.format(i) for i in range(1,600)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_opened_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 200 opened eye images to validation_opened_dir
fnames = ['opened.{}.jpg'.format(i) for i in range(600, 800)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_opened_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 200 opened eye images to test_opened_dir
fnames = ['opened.{}.jpg'.format(i) for i in range(800, 1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_opened_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 200 closed eye images to train_closed_dir
fnames = ['closed.{}.jpg'.format(i) for i in range(1,600)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_closed_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 200 closed eye images to validation_closed_dir
fnames = ['closed.{}.jpg'.format(i) for i in range(600, 800)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_closed_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 200 closed eye images to test_closed_dir
fnames = ['closed.{}.jpg'.format(i) for i in range(800, 1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_closed_dir, fname)
    shutil.copyfile(src, dst)

The lines of code listed above were used to create a new director containing the folders for the train, test and validation images. 600 images were used as the training set while 200 images were used for both the test and validation images. This means that for all the folders train, test and validation, 200 images were placed with eyes opened and closed respectively. 

# 3. Exploratory Data Analysis

To be able to do the the project, we must first know the details about the data. The details are the following:
1. Each of the folders used in the project (train, test, validation) contains 600,200,200 images for each eye state (closed or opend).
2. The image size if 24x24 pixel.
3. The images are of Grayscale format. The reason for this is that the grayscale channel will result to lesser number of parameters to be used for the modeling later as it will produce only 1 channel.
4. Sample images are illustrated below.

<img src="eyes_focus.jpg" width="700">

# 4. Building the Neural Network

Using Tensorflow, we shall be building a neural network with input sizes according to the pixel size of the eye patches and of grayscale format resulting to a **24 x 24 x 1** input size. The approach for the model building is a **binary classification** approach thus the last layer activation function shall be a **sigmoid** function. The depth of the feature maps used initially started at size 32 up to 128 thus it is expected that the size of the feature maps to decrease.

In [2]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24, 24, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 128)        

With this, we can observed that there are a total of 109,313 total parameters for the model. For model compilation, since we've used the sigmoid as the last layer activation function for the binary classification approach, we shall therefore resort to the loss function of **binary_crossentropy** with the optimizer being the **RMSprop optimizer**. The following table shows what loss function to use with regards to the approach and the last layer activation function.

<img src="cheat_sheet_error_function1.png" width="700">

In [4]:
# from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
optimizer=keras.optimizers.RMSprop(lr=1e-4),
metrics=['acc'])

# 5. Data Pre-Processing

After building the neural network, it is now time to pre-process the data into floating point tensors before we can feed them into the model. The pre-processing steps to be used in this project includes the following:

1. Reading the image files.   
2. Decoding the JPEG content to RGB/grayscale grids of pixels.   
3. Converting these into floating-point tensors.   
4. Rescaling the pixel values (between 0 and 255) to the [0, 1] interval.

The main function for this is the **ImageDataGenerator** class from keras which enables us to create the necessary Python generators which shall convert the image files into preprocessed tensors. Since we've used a relatively small dataset, the model is prone to overfitting. This means that the model will have trouble generalizing to new input data. With this, we do **Data Augmentation** which allows the generation of more training data from the existing training image data through random transformations. With this, the model shall be exposed to more visual aspects of the images in order for it improve its generalization. Some of the more commonly used data augmentation techniques include rotating, shifting, flipping and shearing the images. Take note also that we are converting the images to a color mode of grayscale in this section as compliant to the input size to the model we've built earlier.


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,    
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(24, 24), 
    batch_size=20,
    color_mode='grayscale',
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(24, 24), 
    batch_size=20, 
    color_mode='grayscale',
    class_mode='binary')

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

# 6. Modeling

The modeling stage involves using the generators from the training and validation sets. The epoch to be used is 250 with 100 steps per epoch and 50 validation steps. The last line saves the resulting model into a folder named 'models'. The actual runtime of this as of the time of writing is at 35 mins.

In [ ]:
history = model.fit_generator(
                train_generator, 
                validation_data=validation_generator,
                epochs=250, 
                steps_per_epoch=100, 
                validation_steps=50)

model.save('models/model_new4.h5', overwrite=True)

# 7. Analysis of Results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

At first, the model seems to be underfitting however as the number of epoch and the longer the training time, I have observed that the model generalizes better. With this, the purpose of 250 epochs is for the conservation of time as after 250 epochs the model's generalization has improved.

<img src="training_results.jpg" width="400">

# 8. Real-Time Classification using OpenCV

Now that we have a trained model which can accurately classify if a person's eyes is either opened or closed, we can then use it to do real-time processing through video classification of whether a person is actually drowsy or not. Through a python library called **OpenCV**, we can use computer vision techniques and algorithms to correctly detect the needed features of objects that we want to process via webcam feed. 

For this project, we want to first detect the face nd then detect the eyes of the subject. Using a pre-trained classifier called **Haar Cascade Classifier**, we can detect the face of the person in front of the webcam. According to the official documentation and research paper by Paul Viola and Michael Jones entitled "Rapid Object Detection using a Boosted Cascade of Simple Features" in 2001, Haar Cascade is a machine learning algorithm where a cascade function is trained from a set of positive (object to be detected is present) and negative (object to be detected is absent). The idea of Haar cascade is extracting features from images using a kind of ‘filter’ called Haar features, similar to the concept of the convolutional kernel. The idea is passing these Haar features on the image, inspecting one window at the time. Then, for each window, all the pixel intensities of  white and black portions are summed. Finally, the value obtained by subtracting those two summations is the value of the feature extracted. Ideally, a great value of a feature means it is relevant.

For this project, we shall use Haar cascade classifiers for the face, the left eye and the right eyes as we want to first detect the angle of the face and then the orientation of the eyes.

<img src="haar.jpg" width="700">

The bulk of the discussion on Haar Cascade Classifier is highly documented and good resource can be found here: https://medium.com/dataseries/face-recognition-with-opencv-haar-cascade-a289b6ff042a

In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer   #For Alerts
import time


mixer.init()
sound = mixer.Sound('alarm.wav') #For alerting the user in case of drowsiness

# Load Haar Cascade Classifiers
face = cv2.CascadeClassifier(
    'haar cascade files\haarcascade_frontalface_alt.xml') 
leye = cv2.CascadeClassifier(
    'haar cascade files\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier(
    'haar cascade files\haarcascade_righteye_2splits.xml')

Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
lbl = ['Close', 'Open']   #List of Eye States

model = load_model('models/model_new2.h5')   #Load the trained Model earlier
path = os.getcwd()
cap = cv2.VideoCapture(0)     #Instantiates camera feed from the webcam
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count = 0   #Initial Count Value for incrementation
score = 0   #Initial Score Count for drowsiness threshold
thicc = 2
rpred = [99]  #Placeholders
lpred = [99]

A function from the code below called **face.detectMultiScale()** returns an array of the face detections with x,y coordinates, and height, the width of the boundary box of the detected face object. This is stored in the variable **faces**.

For the detection of the right and left eyes, the process is the same as the detection of the face. Using the Haar classifiers for the left and right eyes respectively, we again called the method **leye.detectMultiScale() and reye.detectMultiScale()** for the detection of the left and right eye. This time around we are not going to create a boundary box but instead we are going to perform a set of transformations and use the model for the classification task. Transformations include resizing to 24x24, converting the input frames to grayscale, normalization of pixels, and expansion of dimensions for ease of classification. Then we call **model.predict_classes** for classification of eye state.

A counter was also created with a threshold of 15. This simulates the time in seconds of when a person has fallen asleep for 15 seconds. This is then considered as a drowsy state. With this, the alert shall be prompted hopefully waking up and alerting the user. Once the user has been identified as having woke up from a drowsy state, the counter shall decrease the score assigned to it thus removing the alert. The process goes on in real time without any human supervision.

In [4]:
while(True):
    ret, frame = cap.read()     # will read each frame and stores the image in a frame variable.
    height, width = frame.shape[:2]

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   #Convertion of input frames to grayscale
#     print(gray)
    faces = face.detectMultiScale(
        gray, minNeighbors=5, scaleFactor=1.1, minSize=(25, 25)) #Face Detection 
    left_eye = leye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0, height-50), (200, height),
                  (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (100, 100, 100), 1)

    for (x, y, w, h) in right_eye:
        r_eye = frame[y:y+h, x:x+w]
        count = count+1
        r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye, (24, 24))
        r_eye = r_eye/255
        r_eye = r_eye.reshape(24, 24, -1)
        r_eye = np.expand_dims(r_eye, axis=0)
        rpred = model.predict_classes(r_eye)
        if(rpred[0] == 1):
            lbl = 'Open'
        if(rpred[0] == 0):
            lbl = 'Closed'
        break

    for (x, y, w, h) in left_eye:
        l_eye = frame[y:y+h, x:x+w]
        count = count+1
        l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye, (24, 24))
        l_eye = l_eye/255
        l_eye = l_eye.reshape(24, 24, -1)
        l_eye = np.expand_dims(l_eye, axis=0)
        lpred = model.predict_classes(l_eye)
        if(lpred[0] == 1):
            lbl = 'Open'
        if(lpred[0] == 0):
            lbl = 'Closed'
        break
    
    # Drowsiness Score Counter with a threshold of 15 for drowsy. 
    # Alerts the user if score exceeds 15
    if(rpred[0] == 0 and lpred[0] == 0):
        score = score+1
        cv2.putText(frame, "Closed", (10, height-20), font,
                    1, (255, 255, 255), 1, cv2.LINE_AA)
    else:
        score = score-1
        cv2.putText(frame, "Open", (10, height-20), font,
                    1, (255, 255, 255), 1, cv2.LINE_AA)

    if(score < 0):
        score = 0
    cv2.putText(frame, 'Score:'+str(score), (100, height-20),
                font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    if(score > 15):
        # person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path, 'image.jpg'), frame)
        try:
            sound.play()

        except:  # isplaying = False
            pass
        
        #This is for the red box that appears when the user is drowsy
        #indicating that the user has to be alerted
        if(thicc < 16):
            thicc = thicc+2
        else:
            thicc = thicc-2
            if(thicc < 2):
                thicc = 2
        cv2.rectangle(frame, (0, 0), (width, height), (0, 0, 255), thicc)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### Important Note:
The ideal way of running the above code is through the command line as it the current version of the code made by me does not destroy the GUI popup once the user closes the interface. Therefore, a separate notebook was created named "**Individual Project_OpenCV.py**" which was meant to be run on the anaconda command line interface with the argument ***python "Individual Project_OpenCV.py"*** given the assumption that you are working on the current directory on the Anaconda Prompt.

In [5]:
from IPython.display import HTML, display

Below depicts two scenarios from which the project was tested. The first scenario is when the user is in a dimly lit enviroment with his/her glasses on. The second scenario is when the user is also in a dimly lit environment without his/her glasses on. The main idea behind these tests is to know if the project will work on different environments with object obstructions to the eyes such as an eyeglass.

In [6]:
display(HTML("<table><tr><td><img src='image1.gif'></td><td><img src='image2.gif'></td></tr></table>"))

,


# 7. Summary and Conclusion 

To give a summary of what we've accomplished on this project, let us first give emphasis on the two python libraries that made this possible. The first library is Tensorflow and through this, we've created a convolutional neural network to which we've trained to do image classification on the state of the eye whether it is opened or closed. The neural network model achieved a 95% average test accuracy across 250 epochs. The second python library is OpenCV to which we've used the trained model to provide real-time classification of eye state through video inputs. The goal of the project is to detect drowsiness which will serve to reduce drowsy driving and unproductivity due to drowsiness at the workplace. By preventing the occurence of these scenarios, the project will surely help in the reduction of events related to drowsiness.

The project however has its own limitations. The first limitation is the hardware constraints since this was only using a builtin webcam. Newer and particularly higher end webcams can provide far better results as the frame rates can be better. The second limitation is that since the CNN model assumes that it is trained on a singular user, the project is thus limited to only one person/user at a time. The third is that the availability of sample data is somewhat limited, therefore poses more room for improvement.

# 8. Recommendations

* Use better hardware such as a higher end PC or a better camera as input
* Train the model to more images.
* Train the model to let it recognize many people at the same time
* Train the model to avoid ethnicity related issues (e.g. asians have smaller eye dilation)

# 9. References and Acknowledgements

* Drowsy Driving: Asleep at the Wheel. (2019, November 7). Retrieved from Center for Disease Control and Prevention: https://www.cdc.gov/features/dsdrowsydriving/index.html
* Gabriel, E. (2018, February 8). Drowsy driving is a factor in almost 10% of crashes, study finds. Retrieved from CNN Heatlh: https://edition.cnn.com/2018/02/08/health/drowsy-driving-crashes-study/index.html
* Jupyter Notebooks on Convolutional Neural Networks from Professor Chris Monterola
* Deep learning insights and guidance from Professor Erika Legara